In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import argparse
from scipy import interpolate
import tqdm
import nmmn.lsd, nmmn.misc
import os.path
import pandas as pd
import astropy.visualization

In [3]:
# Constants, definitions and units

# many are related to grmonty, but we can set a black hole mass if we want to have a "feeling" for the physical values

global MP, ME, CL, GNEWT, KBOL, SIGMA_THOMSON, MSUN, LSUN, YEAR, MBH
global TPTE_DISK, TPTE_JET, THETAE_MAX
global M_unit, L_unit, T_unit, RHO_unit, U_unit, B_unit, Ne_unit

# all constants in cgs units
ME = 9.1093826e-28 # electron mass
MP = 1.67262171e-24 # proton mass
CL = 2.99792458e10 # speed of light
GNEWT = 6.6742e-8 # gravitational constant
KBOL = 1.3806505e-16 # Boltzmann constant
SIGMA_THOMSON = 0.665245873e-24 # Thomson cross-section
MSUN = 1.989e33 # solar mass
LSUN = 3.827e33 # solar luminosity
YEAR = 31536000 # seconds in a year

# temperature and beta-prescription (Mościbrodzka 2016)
TPTE_DISK = 20. # R_high
TPTE_JET = 1. # R_low
THETAE_MAX = 1000.
TP_OVER_TE = 100.0

sgra=1
m87=0
blazar=0
# grmonty units and BH mass
if (sgra):
    MBH = 4.5e6 * MSUN
    M_unit = 1.0e19
elif (m87):
    MBH = 6.2e9 * MSUN
    M_unit = 1.0e29

L_unit = GNEWT * MBH / (CL * CL)
T_unit = L_unit / CL
RHO_unit = M_unit / (L_unit*L_unit*L_unit)
U_unit = RHO_unit * CL * CL
B_unit = CL * np.sqrt(4. * np.pi * RHO_unit)
Ne_unit = RHO_unit / (MP + ME)

#MBH = 5.0e9 * MSUN
#M_unit = 2.0*10e11

In [4]:
%run -i harm_script2.py

In [5]:
rg("gdump")

In [6]:
runname = "Be105"
                
dumpmin, dumpmax = 225,940
ilim=821+1
#imin = 800
#imax = imin+400
for dumpnumber in range(dumpmin, dumpmax+1):
    
    dumpname = "dump"+str(dumpnumber)
    rd(dumpname)
    N1, N2, N3 = nx, ny, nz
    sigma = bsq/rho
    betapl = 2.0*pg/bsq
    Bphi = B[3]*B_unit
    sigma2d = sigma[:,:,0]
    r2d = (r*np.sin(h))[:,:,0]
    z2d = (r*np.cos(h))[:,:,0]
    
    sigmaext_disc = []
    betaext_disc = []
    BrBphiext_disc = []
    BthBphiext_disc = []
    #sigmaextcenter_disc = []

    sigmaext_jet = []
    betaext_jet = []
    BrBphiext_jet = []
    BthBphiext_jet = []
    #sigmaextcenter_jet = []
       
    for i in range(0,ilim):
        for j in range(0,N2):
            for k in range(0,N3):
                myfilename_disc = "/work/gustavo/gyst/sheets_disc/dump%03d_%d_%d_%d_s" %(dumpnumber,i,j,k)
                myfilename_jet = "/work/gustavo/gyst/sheets_jet/dump%03d_%d_%d_%d_s" %(dumpnumber,i,j,k)
                if (os.path.isfile(myfilename_disc)):

                    myfile_disc = open(myfilename_disc, "rb")
                    dtype = np.float64
                    body_disc = np.fromfile(myfile_disc,dtype=dtype,count=-1)
                    ind2_disc = int((len(body_disc))/10)
                    mydata_disc = body_disc.view().reshape((10,ind2_disc), order='F')
                    
                    myBr_disc       = mydata_disc[0]
                    myBth_disc      = mydata_disc[1]
                    myBphi_disc     = mydata_disc[2]
                    myJ_disc        = mydata_disc[3]
                    mybeta_disc     = mydata_disc[4]
                    mysigma_disc    = mydata_disc[5]
                    mysigmaphi_disc = mydata_disc[6]
                    myi_disc        = mydata_disc[7]
                    myj_disc        = mydata_disc[8]
                    myk_disc        = mydata_disc[9]
                    myfile_disc.close()

                    sigmaext_disc.append(mysigma_disc[0])
                    sigmaext_disc.append(mysigma_disc[-1])
                    #betaext_disc.append(mybeta_disc[0])
                    #betaext_disc.append(mybeta_disc[-1])
                    #sigmaextcenter_disc.append(sigma[i][j][k])
                    BrBphiext_disc.append(abs(B[1][i][j][k]/B[3][i][j][k]))
                    BthBphiext_disc.append(abs(B[2][i][j][k]/B[3][i][j][k]))
                    
                if (os.path.isfile(myfilename_jet)):

                    myfile_jet = open(myfilename_jet, "rb")
                    dtype = np.float64
                    body_jet = np.fromfile(myfile_jet,dtype=dtype,count=-1)
                    ind2_jet = int((len(body_jet))/10)
                    mydata_jet = body_jet.view().reshape((10,ind2_jet), order='F')
                    
                    myBr_jet      = mydata_jet[0]
                    myBth_jet     = mydata_jet[1]
                    myBphi_jet    = mydata_jet[2]
                    myJ_jet       = mydata_jet[3]
                    mybeta_jet    = mydata_jet[4]
                    mysigma_jet   = mydata_jet[5]
                    mysigmaphi_jet = mydata_jet[6]
                    myi_jet        = mydata_jet[7]
                    myj_jet        = mydata_jet[8]
                    myk_jet        = mydata_jet[9]
                    myfile_jet.close()

                    sigmaext_jet.append(mysigma_jet[0])
                    sigmaext_jet.append(mysigma_jet[-1])
                    #betaext_jet.append(mybeta_jet[0])
                    #betaext_jet.append(mybeta_jet[-1])
                    #sigmaextcenter_jet.append(sigma[i][j][k])
                    BrBphiext_jet.append(abs(B[1][i][j][k]/B[3][i][j][k]))
                    BthBphiext_jet.append(abs(B[2][i][j][k]/B[3][i][j][k]))
    
    sigma_disc = []
    sigma_disc = sigmaext_disc
    sigma_jet = []
    sigma_jet = sigmaext_jet
    #sigmacenter_disc = []
    #sigmacenter_disc = sigmaextcenter_disc
    #sigmacenter_jet = []
    #sigmacenter_jet = sigmaextcenter_jet
    Br_disc = []
    Br_disc = BrBphiext_disc
    Br_jet = []
    Br_jet = BrBphiext_jet
    Bth_disc = []
    Bth_disc = BthBphiext_disc
    Bth_jet = []
    Bth_jet = BthBphiext_jet
    
    sigma_disc = [x for x in sigma_disc if str(x) != 'nan']
    sigma_disc = [x for x in sigma_disc if str(x) != 'inf']
    sigma_disc = np.array(sigma_disc)
    sigma_jet = [x for x in sigma_jet if str(x) != 'nan']
    sigma_jet = [x for x in sigma_jet if str(x) != 'inf']
    sigma_jet = np.array(sigma_jet)
    
    #sigmacenter_disc = [x for x in sigmacenter_disc if str(x) != 'nan']
    #sigmacenter_disc = [x for x in sigmacenter_disc if str(x) != 'inf']
    #sigmacenter_disc = np.array(sigmacenter_disc)
    #sigmacenter_jet = [x for x in sigmacenter_jet if str(x) != 'nan']
    #sigmacenter_jet = [x for x in sigmacenter_jet if str(x) != 'inf']
    #sigmacenter_jet = np.array(sigmacenter_jet)
    
    Br_disc = [x for x in Br_disc if str(x) != 'nan']
    Br_disc = [x for x in Br_disc if str(x) != 'inf']
    Br_disc = np.array(Br_disc)
    Br_jet = [x for x in Br_jet if str(x) != 'nan']
    Br_jet = [x for x in Br_jet if str(x) != 'inf']
    Br_jet = np.array(Br_jet)
    
    Bth_disc = [x for x in Bth_disc if str(x) != 'nan']
    Bth_disc = [x for x in Bth_disc if str(x) != 'inf']
    Bth_disc = np.array(Bth_disc)
    Bth_jet = [x for x in Bth_jet if str(x) != 'nan']
    Bth_jet = [x for x in Bth_jet if str(x) != 'inf']
    Bth_jet = np.array(Bth_jet)
    
    
    Jfile_j = "/work/gustavo/gyst/jcs_files_jet/"+dumpname+"_jcs"
    Jfilechar_j = "/work/gustavo/gyst/jcs_files_jet/"+dumpname+"_jcs_char"
    Jfile_d = "/work/gustavo/gyst/jcs_files_disc/"+dumpname+"_jcs"
    Jfilechar_d = "/work/gustavo/gyst/jcs_files_disc/"+dumpname+"_jcs_char"

    f1_j = open(Jfile_j, "rb")
    dtype = np.float64
    body_j = np.fromfile(f1_j,dtype=dtype,count=-1)
    JJ_cs_j = body_j.view().reshape((int(N1),int(N2),int(N3)), order='C')

    f2_j = open(Jfilechar_j, "rb")
    dtype = np.float64
    body_j = np.fromfile(f2_j,dtype=dtype,count=-1)
    JJ_cs_char_j = body_j.view().reshape((int(N1),int(N2),int(N3)), order='C')

    f1_d = open(Jfile_d, "rb")
    dtype = np.float64
    body_d = np.fromfile(f1_d,dtype=dtype,count=-1)
    JJ_cs_d = body_d.view().reshape((int(N1),int(N2),int(N3)), order='C')

    f2_d = open(Jfilechar_d, "rb")
    dtype = np.float64
    body_d = np.fromfile(f2_d,dtype=dtype,count=-1)
    JJ_cs_char_d = body_d.view().reshape((int(N1),int(N2),int(N3)), order='C')

    JJ_cs_char_ind_j = numpy.zeros(shape=(N1,N2,N3))
    JJ_cs_char_ind_d = numpy.zeros(shape=(N1,N2,N3))
    for i in range(N1):
        for j in range(N2):
            for k in range(N3):
                if JJ_cs_char_j[i][j][k] > 0:
                    JJ_cs_char_ind_j[i][j][k] = 1
                if JJ_cs_char_d[i][j][k] > 0:
                    JJ_cs_char_ind_d[i][j][k] = 1

    rx_j = []
    ry_j = []
    rx_d = []
    ry_d = []
    for i in range(525):
        for j in range(N2):
            for k in range(N3):
                if (JJ_cs_char_ind_j[i][j][k] == 1):
                    rx_j.append(r[i][j][k] * np.sin(h[i][j][k]))
                    ry_j.append(r[i][j][k] * np.cos(h[i][j][k]))
                if (JJ_cs_char_ind_d[i][j][k] == 1):
                    rx_d.append(r[i][j][k] * np.sin(h[i][j][k]))
                    ry_d.append(r[i][j][k] * np.cos(h[i][j][k]))


    hfont = {'fontname':'Helvetica'}
    fig = plt.figure(figsize=[12.8, 7.2])
    #fig = plt.figure(figsize=[12.8, 14.4])
    #fig = plt.figure(figsize=[6.4+3.2, 4.8])
    gs = GridSpec(3,3)
    mycolor_d = "blue"
    mycolor_j = "red"
    latexsize = 18
    fontsize = 13
    
    mirror = 1
    if(mirror):
        xmin = 0
        xmax = 50
        ymin = -xmax
        ymax = xmax
        step = xmax/2
    else:
        xmin = 0
        xmax = 10
        ymin = 0
        ymax = 20
        step = 5
    
    ax1 = plt.subplot(gs[:,0])
    aphi=psicalc()
    amax = aphi.max()
    cs1,cb1 = plc(np.log10(rho),
                  levels=np.linspace(-8, 0, 100),
                  isfilled=1, 
                  k=0, 
                  xy=1, 
                  xmax=xmax,
                  ymax=ymax, 
                  dobh=1, 
                  cb=True, 
                  extend = "both", 
                  pretty=1,
                  fntsize=fontsize)
    plt.contour(r2d, z2d, np.log10(sigma2d), levels=[0], colors=('xkcd:yellow'), linestyles=('-.'), linewidths=(0.8))
    ax1 = plt.gca()
    #ax1.set_xlabel(r"$R\ [r_g]$",fontsize=latexsize,labelpad=-5, **hfont)
    ax1.set_ylabel(r"$z\ [r_g]$",fontsize=latexsize,labelpad=-5, **hfont)
    cb1.ax.set_xlabel(r"$\log\;\rho$",fontsize=latexsize,ha="left")
    cb1.set_ticks([-8,-7,-6,-5,-4,-3,-2,-1,0])
    if(mirror):
        ax1.set_xticks(np.arange(xmin, xmax+1, step=step)) #GS
        ax1.set_yticks(np.arange(ymin, ymax+1, step=step)) #GS
        ax1.tick_params(axis="x", labelsize=fontsize)
        ax1.tick_params(axis="y", labelsize=fontsize)
    else:
        plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
        plt.yticks(np.arange(0, ymax+1, step=step), **hfont) #GS
        plt.xlim(0,xmax+1)
        plt.ylim(0,ymax+1)
    plc(aphi,levels=np.linspace(-amax,amax,20)[1:-1],colors="xkcd:black",linewidths=0.5,xy=-1)
    ax1.set_title(r"$t=%.5g$"%np.round(t)+r"$\;r_g/c$", fontsize=latexsize)
    
    
    ax2 = plt.subplot(gs[:,1])
    cs2,cb2 = plc(np.log10(sigma), 
                  levels=np.linspace(-5,2, 100),
                  isfilled=1, 
                  k=0, 
                  xy=1, 
                  xmax=xmax, 
                  ymax=ymax, 
                  dobh=1, 
                  cb=True, 
                  extend = "both", 
                  pretty=1, 
                  #cmap='magma',
                  cmap='gray',
                  fntsize=fontsize)
    plt.scatter(rx_d, ry_d, c =mycolor_d, marker = 'o', s = 2.5)#, markersize = 0.01)
    plt.scatter(rx_j, ry_j, c =mycolor_j, marker = 'o', s = 2.5)#, markersize = 0.01)
    plt.contour(r2d, z2d, np.log10(sigma2d), levels=[0], colors=('xkcd:yellow'), linestyles=('-.'), linewidths=(0.8))
    ax2 = plt.gca()
    ax2.set_xlabel(r"$R\ [r_g]$",fontsize=latexsize,labelpad=-5)
    #ax6.set_ylabel(r"$z\ [r_g]$",fontsize=latexsize,labelpad=-5)
    cb2.ax.set_xlabel(r"$\log\;\sigma$",fontsize=latexsize, labelpad=1, ha="left")
    cb2.set_ticks([-5,-4,-3,-2,-1,0,1,2])
    if(mirror):
        ax2.set_xticks(np.arange(xmin, xmax+1, step=step)) #GS
        ax2.set_yticks(np.arange(ymin, ymax+1, step=step)) #GS
        ax2.tick_params(axis="x", labelsize=fontsize)
        ax2.tick_params(axis="y", labelsize=fontsize)
    else:
        plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
        plt.yticks(np.arange(0, ymax+1, step=step), **hfont) #GS
        plt.xlim(0,xmax+1)
        plt.ylim(0,ymax+1)
    #ax2.set_title(r"$t=%.5g$"%np.round(t)+r"$\;r_g/c$", fontsize=latexsize)
    ax2.set_title("SITES", fontsize=latexsize)
    
   
    ax3 = plt.subplot(gs[0,2])
    plt.hist(np.log10(Br_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(Br_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log(B_r/B_\phi)$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)

    ax4 = plt.subplot(gs[1,2])
    plt.hist(np.log10(Bth_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(Bth_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log(B_\theta/B_\phi)$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)

    ax5 = plt.subplot(gs[2,2])
    plt.hist(np.log10(sigma_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(sigma_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log\;\sigma$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    
    

    

    
    plt.tight_layout()
    
    plt.savefig("/work/gustavo/gyst/test_sites_jet/"+dumpname+"_testsites.png", dpi=120)
    plt.clf()
    #print(sigma_jet)
    #print(sigma_disc)
    """
    hfont = {'fontname':'Helvetica'}
    fig = plt.figure(figsize=[12.8, 7.2])
    #fig = plt.figure(figsize=[12.8, 14.4])
    #fig = plt.figure(figsize=[6.4+3.2, 4.8])
    gs = GridSpec(3,3)
    mycolor_d = "blue"
    mycolor_j = "red"
    latexsize = 18
    fontsize = 13
    
    mirror = 1
    if(mirror):
        xmin = 0
        xmax = 50
        ymin = -xmax
        ymax = xmax
        step = xmax/2
    else:
        xmin = 0
        xmax = 10
        ymin = 0
        ymax = 20
        step = 5
    
    ax1 = plt.subplot(gs[:,0])
    aphi=psicalc()
    amax = aphi.max()
    cs1,cb1 = plc(np.log10(rho),
                  levels=np.linspace(-8, 0, 100),
                  isfilled=1, 
                  k=0, 
                  xy=1, 
                  xmax=xmax,
                  ymax=ymax, 
                  dobh=1, 
                  cb=True, 
                  extend = "both", 
                  pretty=1,
                  fntsize=fontsize)
    plt.contour(r2d, z2d, np.log10(sigma2d), levels=[0], colors=('xkcd:yellow'), linestyles=('-.'), linewidths=(0.8))
    ax1 = plt.gca()
    #ax1.set_xlabel(r"$R\ [r_g]$",fontsize=latexsize,labelpad=-5, **hfont)
    ax1.set_ylabel(r"$z\ [r_g]$",fontsize=latexsize,labelpad=-5, **hfont)
    cb1.ax.set_xlabel(r"$\log\;\rho$",fontsize=latexsize,ha="left")
    cb1.set_ticks([-8,-7,-6,-5,-4,-3,-2,-1,0])
    if(mirror):
        ax1.set_xticks(np.arange(xmin, xmax+1, step=step)) #GS
        ax1.set_yticks(np.arange(ymin, ymax+1, step=step)) #GS
        ax1.tick_params(axis="x", labelsize=fontsize)
        ax1.tick_params(axis="y", labelsize=fontsize)
    else:
        plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
        plt.yticks(np.arange(0, ymax+1, step=step), **hfont) #GS
        plt.xlim(0,xmax+1)
        plt.ylim(0,ymax+1)
    plc(aphi,levels=np.linspace(-amax,amax,20)[1:-1],colors="xkcd:black",linewidths=0.5,xy=-1)
    ax1.set_title(r"$t=%.5g$"%np.round(t)+r"$\;r_g/c$", fontsize=latexsize)
    
    
    ax2 = plt.subplot(gs[:,1])
    cs2,cb2 = plc(np.log10(sigma), 
                  levels=np.linspace(-5,2, 100),
                  isfilled=1, 
                  k=0, 
                  xy=1, 
                  xmax=xmax, 
                  ymax=ymax, 
                  dobh=1, 
                  cb=True, 
                  extend = "both", 
                  pretty=1, 
                  #cmap='magma',
                  cmap='gray',
                  fntsize=fontsize)
    plt.scatter(rx_d, ry_d, c =mycolor_d, marker = 'o', s = 2.5)#, markersize = 0.01)
    plt.scatter(rx_j, ry_j, c =mycolor_j, marker = 'o', s = 2.5)#, markersize = 0.01)
    plt.contour(r2d, z2d, np.log10(sigma2d), levels=[0], colors=('xkcd:yellow'), linestyles=('-.'), linewidths=(0.8))
    ax2 = plt.gca()
    ax2.set_xlabel(r"$R\ [r_g]$",fontsize=latexsize,labelpad=-5)
    #ax6.set_ylabel(r"$z\ [r_g]$",fontsize=latexsize,labelpad=-5)
    cb2.ax.set_xlabel(r"$\log\;\sigma$",fontsize=latexsize, labelpad=1, ha="left")
    cb2.set_ticks([-5,-4,-3,-2,-1,0,1,2])
    if(mirror):
        ax2.set_xticks(np.arange(xmin, xmax+1, step=step)) #GS
        ax2.set_yticks(np.arange(ymin, ymax+1, step=step)) #GS
        ax2.tick_params(axis="x", labelsize=fontsize)
        ax2.tick_params(axis="y", labelsize=fontsize)
    else:
        plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
        plt.yticks(np.arange(0, ymax+1, step=step), **hfont) #GS
        plt.xlim(0,xmax+1)
        plt.ylim(0,ymax+1)
    #ax2.set_title(r"$t=%.5g$"%np.round(t)+r"$\;r_g/c$", fontsize=latexsize)
    ax2.set_title("SITES", fontsize=latexsize)
    
   
    ax3 = plt.subplot(gs[0,2])
    plt.hist(np.log10(Br_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(Br_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log(B_r/B_\phi)$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)

    ax4 = plt.subplot(gs[1,2])
    plt.hist(np.log10(Bth_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(Bth_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log(B_\theta/B_\phi)$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)

    ax5 = plt.subplot(gs[2,2])
    plt.hist(np.log10(sigmacenter_disc), bins=20, rwidth=0.9, range=[-5,5], label="D", color=mycolor_d, zorder=2, alpha=0.6)
    plt.hist(np.log10(sigmacenter_jet), bins=20, rwidth=0.9, range=[-5,5], label="J", color=mycolor_j, zorder=2, alpha=0.6)
    #plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
    _, max_ = plt.ylim()
    plt.xlabel(r"$\log\;\sigma$", fontsize=latexsize)
    #plt.ylabel("N", fontsize=latexsize)
    plt.grid(axis='x', alpha=0.5, zorder=1)
    plt.grid(axis='y', alpha=0.5, zorder=1)
    plt.legend(loc="best")
    plt.xticks([-5,0,5],fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    
    plt.tight_layout()
    
    plt.savefig("/work/gustavo/gyst/test_sites/"+dumpname+"_testsites_center.png", dpi=120)
    plt.clf()
"""

harm_script2.py:738: RuntimeWarning: invalid value encountered in sqrt
  jcurr = np.sqrt(jsq)
/home/gustavo/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/gustavo/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/gustavo/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

...

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

<Figure size 921.6x518.4 with 0 Axes>

In [15]:
sigmacenter_jet

array([], dtype=float64)

# SANDBOX